In [1]:
import pandas as pd
import os
import glob
import numpy as np

# Attributes Analysis

In [2]:
pd.set_option('display.max_colwidth', 1000)

In [8]:
all_data_path = '/Users/sonynka/HTW/IC/data/fashionid'
df = pd.read_csv(os.path.join(all_data_path, 'data.csv'), sep=';', encoding='utf-8')

## Tags
Analyse most used tags

In [9]:
df.describe()

,attributes,category,color,img_path,img_url,product_name
count,3194,3194,3194,3194,3194,3194
unique,3194,1,8,3194,3194,3051
top,"Damen Kleid von MARC O'POLO,Reine Baumwolle,Ausgestellter Schnitt,Abgerundeter V-Ausschnitt,Dreiviertel-Ärmel,Dekorative Teilungsnaht auf der Rückseite,Taillengürtel zum Binden,Rückenlänge bei Größe 36: 112 cm,Unser Model ist 177 cm groß und trägt Größe 36.,Farbe Schwarz,Artikelnummer: 9773154",kleider,blue,kleider/9758377.jpg,"https://img.fidcdn.net/r17/product/wiesnkoenig-dirndl-mit-schuerze-aus-schimmernder-floraler-spitze-schokobraun_9318687,3874f7.jpg",Montego Kleid mit floralem Muster - Marineblau
freq,1,3194,1062,1,1,7


In [10]:
df['tags'] = df['attributes'].apply(lambda x: x.lower().split(','))

In [11]:
tag_count = {}
def count_tags(tags):
    for tag in tags:
        t = tag.lower()
        if t in tag_count:
            tag_count[t] += 1
        else:
            tag_count[t] = 1
            
a = df['tags'].apply(count_tags)

In [12]:
tag_df = pd.Series(tag_count).reset_index().rename(columns={'index': 'tag', 0: 'count'})
num_imgs = df.shape[0]
tag_df['ratio'] = (pd.to_numeric(tag_df['count']) / num_imgs * 100).round(2)
tag_df = tag_df[tag_df['ratio'] > 1]
tag_df = tag_df.sort_values('ratio', ascending=False)

In [13]:
tag_df['tag'].unique().tolist()

['rundhalsausschnitt',
 'taillierter schnitt',
 'unser model ist 177 cm groß und trägt größe 36.',
 'farbe schwarz',
 'nahtreißverschluss auf der rückseite',
 'unser model ist 177 cm groß.',
 'blickdichtes unterkleid',
 'ausgestellter rockteil',
 'ärmellos',
 'ausgestellter schnitt',
 'reißverschluss auf der rückseite',
 'chiffon',
 'unser model ist 177 cm groß und trägt größe m.',
 'farbe dunkelblau',
 'eng anliegender schnitt',
 'farbe marineblau',
 'v-ausschnitt',
 'haken- und nahtreißverschluss auf der rückseite',
 'geknöpfter schlüssellochausschnitt auf der rückseite',
 'taillierter',
 'dreiviertel-ärmel',
 'florales muster',
 'kurze ärmel',
 'rückenlänge bei größe 36: 95\xa0cm',
 'wattierte cups',
 'leichtes unterkleid',
 ' ärmelloser schnitt',
 'unser model ist 177 cm groß und trägt größe s.',
 'reine baumwolle',
 'reine viskose',
 '5\xa0cm',
 'farbe offwhite',
 'farbe rosé',
 'unser model ist 177\xa0cm groß und trägt größe 36.',
 'rückenlänge bei größe 36: 90\xa0cm',
 'zwei ein

## Select useful attributes
Find attributes that are used a lot and can be useful for the application.
Create a special column in the dataframe with each of the useful tags.

In [14]:
def create_tag_column(df, tag_list, column_name):
    """ 
        Create a new column in the given dataframe with the specified column name. 
        The column holds the tag if the tag is in the tag_list otherwise it holds np.nan.
    """
    
    df[column_name] = df['tags'].apply(lambda x: [s for s in x if s in tag_list])
    df[column_name] = [x[0].split(':')[-1].strip() if len(x) > 0 else np.nan for x in df[column_name]]
    
    return df

### Ärmellänge

In [15]:
tag_df[tag_df['tag'].str.contains('ärmel')]

,tag,count,ratio
8339,ärmellos,462,14.46
4556,dreiviertel-ärmel,235,7.36
5560,kurze ärmel,232,7.26
211,ärmelloser schnitt,208,6.51
5623,lange ärmel,135,4.23
5395,kappärmel,112,3.51
8342,ärmelloser,81,2.54
5,angeschnittene ärmel,75,2.35
8346,ärmelloses unterkleid,60,1.88
213,ärmelloses unterkleid,41,1.28


In [18]:
aermel_list = tag_df[tag_df['tag'].str.contains('ärmel')]['tag'].tolist()
df = create_tag_column(df, aermel_list, 'ärmellänge')

In [46]:
df['ärmellänge'].unique()

array(['ärmellos', nan, 'langarm', 'viertelarm', 'dreiviertelarm'], dtype=object)

Merging similiar attributes together

In [34]:
df['ärmellänge'] = ['ärmellos' if x == 'ärmelloser schnitt' else x for x in df['ärmellänge']]
df['ärmellänge'] = ['ärmellos' if x == 'ärmelloser' else x for x in df['ärmellänge']]

Renaming attributes according to the aboutyou dataset

In [43]:
df['ärmellänge'] = ['langarm' if x == 'lange ärmel' else x for x in df['ärmellänge']]
df['ärmellänge'] = ['viertelarm' if x == 'kurze ärmel' else x for x in df['ärmellänge']]
df['ärmellänge'] = ['dreiviertelarm' if x == 'dreiviertel-ärmel' else x for x in df['ärmellänge']]
df['ärmellänge'] = ['dreiviertelarm' if x == 'dreiviertel-ärmel mit ausgestellten abschlüssen' else x for x in df['ärmellänge']]

Deleting useless tags

In [45]:
df['ärmellänge'] = [x if x in ['ärmellos', 'langarm', 'viertelarm', 'dreiviertelarm'] else np.nan for x in df['ärmellänge']]

### Muster

In [67]:
tag_df[(tag_df['tag'].str.contains(r'floral|streif|punkt|muster|spitze$'))]

,tag,count,ratio
5054,florales muster,232,7.26
7573,streifenmuster,145,4.54
327,allover-muster,131,4.10
6322,oberteil aus floraler spitze,85,2.66
5005,florale spitze,81,2.54
4998,florale häkelspitze,50,1.57
7518,silikonstreifen für sicheren halt,38,1.19


In [68]:
muster_list = tag_df[(tag_df['tag'].str.contains(r'floral|streif|punkt|muster|spitze$'))]['tag'].tolist()
df = create_tag_column(df, muster_list, 'muster')

In [72]:
df['muster'].unique()

array([nan, 'gestreift', 'geblümt/floral', 'all-over-muster', 'spitze'], dtype=object)

In [70]:
df['muster'] = ['gestreift' if x == 'streifenmuster' else x for x in df['muster']]
df['muster'] = ['geblümt/floral' if x == 'florales muster' else x for x in df['muster']]
df['muster'] = ['all-over-muster' if x == 'allover-muster' else x for x in df['muster']]
df['muster'] = ['spitze' if 'spitze' in str(x) else x for x in df['muster']]

In [71]:
df['muster'] = [x if x in ['gestreift', 'geblümt/floral', 'all-over-muster', 'spitze'] else np.nan for x in df['muster']]

# Create dummies

In [75]:
df = df.set_index('img_url')
df.head()

,attributes,category,color,img_path,product_name,tags,ärmellänge,ausschnitt,muster
img_url,,,,,,,,,
"https://img.fidcdn.net/r17/product/guess-kleid-porzia-mit-einsatz-aus-mesh-schwarz_9765638,3f8d67.jpg","Damen Kleid von GUESS,Eng anliegender, ärmelloser Schnitt,Materialmischung mit Stretch-Anteil,Einsatz aus Mesh,Rundhalsausschnitt,Seitlicher Nahtreißverschluss,Geknöpfter Schlüssellochausschnitt auf der Rückseite,Nieten- und Paillettenbesatz,Logo-Applikation,Rückenlänge bei Größe S. 89 cm,Farbe Schwarz,Artikelnummer: 9765638,Unser Model ist 177 cm groß und trägt Größe S.",kleider,black,kleider/9765638.jpg,Guess Porzia Dress –Kleid 'Porzia' mit Einsatz aus Mesh - Schwarz,"[damen kleid von guess, eng anliegender, ärmelloser schnitt, materialmischung mit stretch-anteil, einsatz aus mesh, rundhalsausschnitt, seitlicher nahtreißverschluss, geknöpfter schlüssellochausschnitt auf der rückseite, nieten- und paillettenbesatz, logo-applikation, rückenlänge bei größe s. 89 cm, farbe schwarz, artikelnummer: 9765638, unser model ist 177 cm groß und trägt größe s.]",ärmellos,rundhals-ausschnitt,NaN
"https://img.fidcdn.net/r17/product/jakes-maxikleid-mit-streifenmuster-schwarz_9805106,bb47d7.jpg","Damen Maxikleid von Jake*s,Baumwollmischung,Ärmelloser, eng anliegender Schnitt,Leichte Struktur,Rundhalsausschnitt,Seitenschlitze,Streifenmuster,Rückenlänge bei Größe S: 129 cm,Farbe Schwarz,Artikelnummer: 9805106,Unser Model ist 177 cm groß und trägt Größe S.",kleider,black,kleider/9805106.jpg,Jake*s Maxikleid mit Streifenmuster - Schwarz,"[damen maxikleid von jake*s, baumwollmischung, ärmelloser, eng anliegender schnitt, leichte struktur, rundhalsausschnitt, seitenschlitze, streifenmuster, rückenlänge bei größe s: 129 cm, farbe schwarz, artikelnummer: 9805106, unser model ist 177 cm groß und trägt größe s.]",ärmellos,rundhals-ausschnitt,gestreift
"https://img.fidcdn.net/r17/product/only-kleid-mit-rockteil-aus-floraler-spitze-schwarz_9815363,3aa47b.jpg","Damen Kleid von ONLY,Baumwoll-Elasthan-Mix,Modell Nala,Ärmelloser, taillierter Schnitt,Rundhalsausschnitt aus floraler Spitze,Abgesetzter, ausgestellter Rockteil aus floraler Spitze,Eingearbeiteter Unterrrock,Rückenlänge bei Größe S: 88 cm,Farbe Schwarz,Artikelnummer: 9815363,Unser Model ist 177 cm groß und trägt Größe S.",kleider,black,kleider/9815363.jpg,Only Kleid mit Rockteil aus floraler Spitze - Schwarz,"[damen kleid von only, baumwoll-elasthan-mix, modell nala, ärmelloser, taillierter schnitt, rundhalsausschnitt aus floraler spitze, abgesetzter, ausgestellter rockteil aus floraler spitze, eingearbeiteter unterrrock, rückenlänge bei größe s: 88 cm, farbe schwarz, artikelnummer: 9815363, unser model ist 177 cm groß und trägt größe s.]",ärmellos,NaN,NaN
"https://img.fidcdn.net/r17/product/only-one-shoulder-kleid-mit-volantbesatz-schwarz_9815368,5ac017.jpg","Damen Kleid von ONLY,Chiffon,Modell Maya,Ausgestellter Schnitt,One-Shoulder-Träger,Innen liegender Silikonstreifen für besseren Halt,Volantbesatz,Eingearbeitetes Unterkleid,Rückenlänge bei Größe 36: 88 cm,Farbe Schwarz,Artikelnummer: 9815368,Unser Model ist 177 cm groß und trägt Größe 36.",kleider,black,kleider/9815368.jpg,Only One-Shoulder-Kleid mit Volantbesatz - Schwarz,"[damen kleid von only, chiffon, modell maya, ausgestellter schnitt, one-shoulder-träger, innen liegender silikonstreifen für besseren halt, volantbesatz, eingearbeitetes unterkleid, rückenlänge bei größe 36: 88 cm, farbe schwarz, artikelnummer: 9815368, unser model ist 177 cm groß und trägt größe 36.]",NaN,NaN,NaN
"https://img.fidcdn.net/r17/product/windsor-etuikleid-mit-taillenguertel-schwarz_9875358,44d04f.jpg","Damen Etuikleid von Windsor,Baumwollmischung mit Stretch-Anteil,Eng anliegender Schnitt,Umgelegter Kragen,Ärmellos,Reißverschluss auf der Rückseite,Drapierung mit gelegten Falten im Taillenbereich,Taillengürtel zum Binden,Rückenlänge bei Größe 36: 100 cm,Farbe Schwarz,Artikelnummer: 9875358,Unser Model ist 177 cm groß und trägt Größe 36.",kleider,black,kleide

In [76]:
df_dum = pd.get_dummies(df[['category', 'color', 'ärmellänge', 'muster']]).reset_index()

In [77]:
df_dum.head()

,img_url,category_kleider,color_beige,color_black,color_blue,color_green,color_pink,color_red,color_white,color_yellow,ärmellänge_dreiviertelarm,ärmellänge_langarm,ärmellänge_viertelarm,ärmellänge_ärmellos,muster_all-over-muster,muster_geblümt/floral,muster_gestreift,muster_spitze
0,"https://img.fidcdn.net/r17/product/guess-kleid-porzia-mit-einsatz-aus-mesh-schwarz_9765638,3f8d67.jpg",1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,"https://img.fidcdn.net/r17/product/jakes-maxikleid-mit-streifenmuster-schwarz_9805106,bb47d7.jpg",1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0
2,"https://img.fidcdn.net/r17/product/only-kleid-mit-rockteil-aus-floraler-spitze-schwarz_9815363,3aa47b.jpg",1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,"https://img.fidcdn.net/r17/product/only-one-shoulder-kleid-mit-volantbesatz-schwarz_9815368,5ac017.jpg",1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"https://img.fidcdn.net/r17/product/windsor-etuikleid-mit-taillenguertel-schwarz_9875358,44d04f.jpg",1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [78]:
df_dum = df_dum.groupby(['img_url']).max().reset_index()

In [79]:
df_dum.head()

,img_url,category_kleider,color_beige,color_black,color_blue,color_green,color_pink,color_red,color_white,color_yellow,ärmellänge_dreiviertelarm,ärmellänge_langarm,ärmellänge_viertelarm,ärmellänge_ärmellos,muster_all-over-muster,muster_geblümt/floral,muster_gestreift,muster_spitze
0,"https://img.fidcdn.net/r17/product/120-lino-kleid-aus-leinen-mit-collierkragen-blau_9767844,1ab631.jpg",1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,"https://img.fidcdn.net/r17/product/120-lino-kleid-aus-leinen-mit-floralem-muster-weiss_9767849,32e3c3.jpg",1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
2,"https://img.fidcdn.net/r17/product/120-lino-kleid-aus-leinen-mit-kurzer-knopfleiste-graphit_9767838,070745.jpg",1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,"https://img.fidcdn.net/r17/product/120-lino-kleid-aus-leinen-mit-spitze-weiss_9767847,6ed4ae.jpg",1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,"https://img.fidcdn.net/r17/product/120-lino-kleid-aus-leinen-mit-zierperlen-und-ziersteinen-hellgrau_9767833,437a74.jpg",1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [80]:
df_paths = df.groupby(['img_url']).first()['img_path'].reset_index()

In [81]:
df_paths.head()

,img_url,img_path
0,"https://img.fidcdn.net/r17/product/120-lino-kleid-aus-leinen-mit-collierkragen-blau_9767844,1ab631.jpg",kleider/9767844.jpg
1,"https://img.fidcdn.net/r17/product/120-lino-kleid-aus-leinen-mit-floralem-muster-weiss_9767849,32e3c3.jpg",kleider/9767849.jpg
2,"https://img.fidcdn.net/r17/product/120-lino-kleid-aus-leinen-mit-kurzer-knopfleiste-graphit_9767838,070745.jpg",kleider/9767838.jpg
3,"https://img.fidcdn.net/r17/product/120-lino-kleid-aus-leinen-mit-spitze-weiss_9767847,6ed4ae.jpg",kleider/9767847.jpg
4,"https://img.fidcdn.net/r17/product/120-lino-kleid-aus-leinen-mit-zierperlen-und-ziersteinen-hellgrau_9767833,437a74.jpg",kleider/9767833.jpg


In [82]:
df_dum = df_paths.merge(df_dum, how='inner').drop(['img_url'], axis=1)

In [83]:
df_dum.head()

,img_path,category_kleider,color_beige,color_black,color_blue,color_green,color_pink,color_red,color_white,color_yellow,ärmellänge_dreiviertelarm,ärmellänge_langarm,ärmellänge_viertelarm,ärmellänge_ärmellos,muster_all-over-muster,muster_geblümt/floral,muster_gestreift,muster_spitze
0,kleider/9767844.jpg,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,kleider/9767849.jpg,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
2,kleider/9767838.jpg,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,kleider/9767847.jpg,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,kleider/9767833.jpg,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [84]:
df_dum.to_csv('/Users/sonynka/HTW/IC/data/fashionid/img_attr.csv', sep='\t', encoding='utf-8', index=False)

# Merge dataset

In [85]:
df_about = pd.read_csv('/Users/sonynka/HTW/IC/data/aboutyou_paging/img_attr.csv', sep='\t', encoding='utf-8')

In [86]:
df_about.head()

,img_path,category_blusen-und-tuniken,category_hosen,category_jacken,category_jeans,category_jumpsuits-und-overalls,category_kleider,category_roecke,category_shirts,category_strick,...,passform_schmale passform,passform_skinny,passform_slimfit,ausschnitt_eingefasster ausschnitt,ausschnitt_rundhals-ausschnitt,ausschnitt_rückenausschnitt,ausschnitt_tiefer ausschnitt/dekolleté,ausschnitt_u-boot-ausschnitt,ausschnitt_v-ausschnitt,ausschnitt_weiter ausschnitt
0,hosen/00005b236646a41122e2af6608f9bc74.jpg,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,kleider/000319a4556c44335bd4a97c8ebaa715.jpg,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,jeans/00036d763919b77400505f6534b7f353.jpg,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,hosen/00058a0b83bd94b4d4b5f12e12b239fe.jpg,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,strick/000610af30d05925ac9cfd0c28c763d6.jpg,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [87]:
df_fashion = df_dum.copy()

In [88]:
df_fashion.head()

,img_path,category_kleider,color_beige,color_black,color_blue,color_green,color_pink,color_red,color_white,color_yellow,ärmellänge_dreiviertelarm,ärmellänge_langarm,ärmellänge_viertelarm,ärmellänge_ärmellos,muster_all-over-muster,muster_geblümt/floral,muster_gestreift,muster_spitze
0,kleider/9767844.jpg,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,kleider/9767849.jpg,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
2,kleider/9767838.jpg,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,kleider/9767847.jpg,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,kleider/9767833.jpg,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [90]:
df_about = df_about[df_fashion.columns]

In [91]:
df_about.head()

,img_path,category_kleider,color_beige,color_black,color_blue,color_green,color_pink,color_red,color_white,color_yellow,ärmellänge_dreiviertelarm,ärmellänge_langarm,ärmellänge_viertelarm,ärmellänge_ärmellos,muster_all-over-muster,muster_geblümt/floral,muster_gestreift,muster_spitze
0,hosen/00005b236646a41122e2af6608f9bc74.jpg,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,kleider/000319a4556c44335bd4a97c8ebaa715.jpg,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,jeans/00036d763919b77400505f6534b7f353.jpg,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,hosen/00058a0b83bd94b4d4b5f12e12b239fe.jpg,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,strick/000610af30d05925ac9cfd0c28c763d6.jpg,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [92]:
df_merge = df_about.append(df_fashion)

In [93]:
df_merge.shape

(42982, 18)

In [95]:
df_merge.tail()

,img_path,category_kleider,color_beige,color_black,color_blue,color_green,color_pink,color_red,color_white,color_yellow,ärmellänge_dreiviertelarm,ärmellänge_langarm,ärmellänge_viertelarm,ärmellänge_ärmellos,muster_all-over-muster,muster_geblümt/floral,muster_gestreift,muster_spitze
3189,kleider/9392239.jpg,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3190,kleider/9464357.jpg,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3191,kleider/9682361.jpg,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3192,kleider/9752153.jpg,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3193,kleider/9655023.jpg,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [96]:
df_merge.to_csv('/Users/sonynka/HTW/IC/data/img_attr_merged.csv', sep='\t', encoding='utf-8', index=False)